In [ ]:
import os
import pandas as pd
import numpy as np
from glob import glob
from tqdm.notebook import tqdm
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
# os.system("tshark -r /home/user/Sarvesh/Baap_Product/AutoML_DL/security/ISCX_Botnet-Training.pcap -T fields -e ip.src -e frame.len -e     ip.proto -E separator=, -E occurrence=f > /home/user/Sarvesh/Baap_Product/AutoML_DL/security/ISCX_Botnet-Training.csv")

In [ ]:
li = []

for filename in glob('MachineLearningCVE/*.csv'):
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

frame = pd.concat(li, axis=0, ignore_index=True)
frame.columns = [i.lstrip() for i in df.columns]

In [ ]:
frame['Label'].value_counts()

In [ ]:
classes = {'BENIGN':'BENIGN','FTP-Patator':'brute_force_attack','SSH-Patator':'brute_force_attack',
           'DoS ':'DoS_attack','Heart':'DoS_attack','Web':'web_attack', 'Infiltration':'infiltration_attack',
           'Bot':'botnet_attack','DDoS':'DDos_attack','PortScan':'PortScan'}
tags = ['BENIGN', 'FTP-Patator', 'SSH-Patator', 'DoS ', 'Web', 'Infiltration', 'Bot', 'DDoS', 'PortScan','Heart']
new_labels = []
for i, r in tqdm(frame.iterrows()):
    for tag in tags:
        if tag in r['Label']:
            new_labels.append(classes[tag])

print(len(new_labels))
frame['Label'] = new_labels

In [ ]:
frame['Flow Packets/s'].value_counts()['Infinity']

In [ ]:
frame = frame[frame['Flow Bytes/s']!='Infinity']
frame = frame[frame['Flow Packets/s']!='Infinity']

In [ ]:
frame = pd.read_csv('./CICIDS2017/combined_data.csv', index_col=False)
frame['Label'].value_counts()


In [ ]:
X = frame.loc[:, frame.columns != 'Label']
y = frame['Label']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
X_train['Label'] = y_train
X_test['Label'] = y_test

In [ ]:
X_train['Label'].value_counts(),X_test['Label'].value_counts()

In [ ]:
X_train.to_csv('/home/user/Sarvesh/Baap_Product/AutoML_DL/security/CICIDS2017/combined_data_train.csv', index=False)
X_test.to_csv('/home/user/Sarvesh/Baap_Product/AutoML_DL/security/CICIDS2017/combined_data_test.csv', index=False)

In [ ]:
# y = frame[target].loc[:100]
# X = frame.loc[:100, frame.columns != target]
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
# scaler = StandardScaler()
# with pd.option_context('expand_frame_repr', False):
#     print (X_train.iloc[:,:5])
# X_std = scaler.fit_transform(X_train)
# X_std = scaler.inverse_transform(X_std)
# X_std = pd.DataFrame(X_std)
# with pd.option_context('expand_frame_repr', False):
#     print (X_std.iloc[:,:5])
# with pd.option_context('expand_frame_repr', False):
#     print (X_test.iloc[:,:5])
# X_std = scaler.transform(X_test)
# X_std = scaler.inverse_transform(X_std)
# X_std = pd.DataFrame(X_std)
# with pd.option_context('expand_frame_repr', False):
#     print (X_std.iloc[:,:5])

In [ ]:
target = 'Label'
y = frame[target]
X = frame.loc[:, frame.columns != target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
scaler = StandardScaler()
X_std = scaler.fit_transform(X_train)

pca = PCA().fit(X_std)
plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlim(0,40,1)
plt.xlabel('Number of components')
plt.ylabel('Cumulative explained variance')

In [ ]:
pca_transformer = PCA(n_components=35)
pca = pca_transformer.fit_transform(X_std)

clf = RandomForestClassifier(max_depth=5, n_jobs=-1,random_state=0)
clf.fit(pca, y_train)

In [ ]:
y_train_pred = clf.predict(pca)

In [ ]:
pred_df = pd.DataFrame(classification_report(y_train, y_train_pred, output_dict=True))
pred_df

In [ ]:
X_std_test = scaler.transform(X_test)
pca_test = pca_transformer.transform(X_std_test)
y_test_pred = clf.predict(pca_test)
y_test_pred

In [ ]:
pred_df = pd.DataFrame(classification_report(y_test, y_test_pred, output_dict=True))
pred_df

In [ ]:
cm = pd.DataFrame(confusion_matrix(y_train, y_train_pred))
cm